## Cleanup of dates extracted from the _Daily Nation_ "Mailbox" column between 1974 – 1978. Source [_Center for Research Libraries_](https://dds.crl.edu/)

These dates were extract from the newspaper collection script: [column_dates.py](https://github.com/ooduor/lettersiterate/blob/master/column_dates.py). The output generated was in form a tab seperated values (TSV) delimited by tab `\t` character but masquerading CSV. The two columns on the output CSV contains the file name prefix such as _dds-90326-page-8_ and extracted date at the top of the page image e.g. _DAILY NATION, Wednesday, January 4, 1978" 7_.

| | | |
|-|-|-|
| dds-90326-page-8	 | DAILY NATION, Wednesday, January 4, 1978" 7 |
| dds-90327-page-8	 | DAILY NATION, Thursday, January 5, 1978 ~ 7 |
| dds-90328-page-8	 | DAILY, NATION, Friday, January 6, 1978. 7 |
| ...	 | ... |

This notebook illustrates the automated steps at cleaning the data and obtaining the dates for approximately 90%.

 * Open the CSV files organized by year
 * Cleanup known vagaries
 * Attempt to parse dates
 * Export CSV with cleanedup dates

### Reading Dates into a dataframe [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html?highlight=read_csv)

* Specifying the delimiter used in the extracting script and handlinng empty dates
* Add column names since the raw CSV has none

In [1]:
import os
import re
import csv
import pandas as pd
from collections import Counter
from spellchecker import SpellChecker

current_directory = os.getcwd()
prj_root = os.path.dirname(current_directory)
spell = SpellChecker()  # loads default word frequency list
proc_year = '1974'

data = pd.read_csv('{}/column_years/{}.csv'.format(prj_root, proc_year), 
                   delimiter='\t', 
                   header=None,
                   na_filter=False,
                   names=['page_image', 'raw_date_extract'])

data.head(5)

,page_image,raw_date_extract
0,dds-89085-page-8,"> DAIEY""NATION, Tidsdby; sanury4, 197437‘"
1,dds-89086-page-8,"DAILY NATION, Wednesday, January 2, 1974. 7"
2,dds-89087-page-8,
3,dds-89088-page-8,"DAILY. NATION, . January. 1974 7"
4,dds-89089-page-8,"DAILY. NATION, Saturday, Sdduary 5, 1974. \7"


### Cleanup typos using [PySpellchecker](https://github.com/barrust/pyspellchecker) and [Dateparser](https://github.com/scrapinghub/dateparser) libraries

1. Automatically corrects spelling errors by the OCR e.g. **raily. nation, monday, feprugry. 19,, 1975 - 7** _becomes_ **daily. nation, monday, february. 19,, 1975 - 7**
* Use additional heuristics to trim and scope for date only. Remove publisher titles and page numbers e.g. **daily. nation, monday, february. 19,, 1975 - 7** _becomes_ **february. 19,, 1975 **
* Parse the trimmed string into a proper date object e.g. **february. 19,, 1975** _becomes_ **1975-02-19**

In [2]:
import dateparser
from datetime import datetime

def words(text): return re.findall(r'\w+', text)
# def words(text): return re.findall(r'[^-,\.\n\s]+', text, flags=re.ASCII) # [^,\s]+ match any text that is not a tab and not a whitespace.

d = [] # dataframe placeholder
for index, row in data.iterrows():
    page_image_name = row['page_image']
    raw_date = str(row['raw_date_extract']).lower()
    
    WORDS = Counter(words(raw_date))
    misspelled = spell.unknown(WORDS)

    auto_corrects  = {}
    for i, word in enumerate(misspelled):
        auto_corrects[word] = spell.correction(word)
    
    # 1. fix spellings and typos
    spell_checked = re.sub(r'\b(%s)\b' % '|'.join(auto_corrects.keys()), lambda m:auto_corrects.get(m.group(1), m.group(1)), raw_date)
    
    # 2. trim to the scope of date members
    try:
        str_out = spell_checked.split(",", 2)[2]
        str_out = str_out.rsplit(" ", 1)[0]
    except IndexError as e:
        str_out = spell_checked
        
    # 3. Create proper date
    datetime_inst = dateparser.parse(str(str_out))
    if isinstance(datetime_inst, datetime):
        date_inst = datetime_inst.date()
        # make year correct when its wrong
        if date_inst.year != proc_year:
            date_inst = date_inst.replace(year=int(proc_year))
        date_str = str(date_inst)
    else:
        date_str = 'NO DATE'    
    
    d.append([page_image_name, date_str, str_out, raw_date])
    
    # print only the first 10
    if index < 10:
        print("%-50s ====> %30s" % (raw_date, str_out))
        

> daiey"nation, tidsdby; sanury4, 197437‘          ====>                               
daily nation, wednesday, january 2, 1974. 7        ====>               january 2, 1974.
                                                   ====>                               
daily. nation, . january. 1974 7                   ====> daily. nation, . january. 1974 7
daily. nation, saturday, sdduary 5, 1974. \7       ====>               sdduary 5, 1974.
daily nation, monday, january 7, 1974 7            ====>                january 7, 1974
daily nation, tuesday, january 8, 1974 7           ====>                january 8, 1974
daily nation, wednesday, january 9, 1974 .7        ====>                january 9, 1974
daily nation, thursday, january 10, 1974 “7        ====>               january 10, 1974
daily nation, friday, january 11, 1974 7           ====>               january 11, 1974


### New dataframe with proper dates

Date-like objects generated above are stored in new column. Impossible cases are marked with 'NO DATE' which will be added manually after verification in source or inference.

The output is a CSV file with an additional column with extracted dates for each newspaper column.

In [3]:
headers = ['page_image_name', 'cleaned_date', 'scoped_date', 'raw_date']
new_data = pd.DataFrame(d, columns=headers)

cleaned_csv = os.path.join(current_directory, f'PROC_{proc_year}.csv')
new_data.to_csv(cleaned_csv, 
                sep='\t', 
                encoding='utf-8', 
                index=False,
                columns = headers)

new_data.head(10)

,page_image_name,cleaned_date,scoped_date,raw_date
0,dds-89085-page-8,NO DATE,,"> daiey""nation, tidsdby; sanury4, 197437‘"
1,dds-89086-page-8,1974-01-02,"january 2, 1974.","daily nation, wednesday, january 2, 1974. 7"
2,dds-89087-page-8,NO DATE,,
3,dds-89088-page-8,NO DATE,"daily. nation, . january. 1974 7","daily. nation, . january. 1974 7"
4,dds-89089-page-8,NO DATE,"sdduary 5, 1974.","daily. nation, saturday, sdduary 5, 1974. \7"
5,dds-89090-page-8,1974-01-07,"january 7, 1974","daily nation, monday, january 7, 1974 7"
6,dds-89091-page-8,1974-01-08,"january 8, 1974","daily nation, tuesday, january 8, 1974 7"
7,dds-89092-page-8,1974-01-09,"january 9, 1974","daily nation, wednesday, january 9, 1974 .7"
8,dds-89093-page-8,1974-01-10,"january 10, 1974","daily nation, thursday, january 10, 1974 “7"
9,dds-89094-page-8,1974-01-11,"january 11, 1974","daily nation, friday, january 11, 1974 7"
